In [45]:
import time

In [46]:
log = open("log", "rb")

In [47]:
def readLine(log):    
    l = log.read(26)        
    time = int.from_bytes(l[0:8],"little")
    addr = int.from_bytes(l[8:16],"little")
    size = int.from_bytes(l[16:24],"little")
    action = l[24]
    return time, addr, size, action

offset_t, addr, size, action = readLine(log)
print(offset_t, offset_a)
log.seek(0)

sizes = [{}]
hist = {}
nextts = 0

while True:
    try:
        time, addr, size, action = readLine(log)
    except:
        break
        
    time = time - offset_t
    if addr not in hist:
        hist[addr] = 0
    if size not in sizes[nextts]:
        sizes[nextts][size] = 0

    if addr == 94246681803456:
        print(size,action)
    if action == 65: # allocate
        sizes[nextts][size] += 1
        hist[addr] = size
    elif action == 70: # free
        size = hist[addr]
        hist[addr] = 0
        #if addr != 0 and size <= 0:
        #    print(f"free {addr} {size}")
        sizes[nextts][size] -= 1
    else:
        print(f"unknown action {action}")
        
    if time > nextts:
        sizes.append({})
        total = 0
        for s in sizes[nextts]:
            total += sizes[nextts][s] * s
            sizes[nextts+1][s] = sizes[nextts][s]
        total = int(total/1024/1024)
        print( f"time {time} : total mem {total} " )
        nextts += 1   

1643385216 94246671397536
time 1 : total mem 50 
time 2 : total mem 50 
time 3 : total mem 50 
time 4 : total mem 78 
time 5 : total mem 77 
time 6 : total mem 81 
time 7 : total mem 87 
time 8 : total mem 82 
time 9 : total mem 82 
time 10 : total mem 83 
time 11 : total mem 83 
time 12 : total mem 96 
time 13 : total mem 97 
time 14 : total mem 95 
time 15 : total mem 87 
time 16 : total mem 101 
time 17 : total mem 86 
time 18 : total mem 88 
time 19 : total mem 101 
time 20 : total mem 95 
time 21 : total mem 89 
time 22 : total mem 103 
time 23 : total mem 103 
time 24 : total mem 90 
time 25 : total mem 90 
time 26 : total mem 90 
time 27 : total mem 90 
time 28 : total mem 90 
time 29 : total mem 97 
time 30 : total mem 108 
time 31 : total mem 92 
time 32 : total mem 103 
time 33 : total mem 112 
time 34 : total mem 94 
time 35 : total mem 95 
time 36 : total mem 94 
time 37 : total mem 95 
time 38 : total mem 115 
time 39 : total mem 95 
time 40 : total mem 94 
time 41 : total

In [ ]:

sizes[-1] = { x: sizes[-1][x] for x in sizes[-1] if sizes[-1][x] > 1 }

keys = list(sizes[0].keys())
keys.sort()


In [ ]:
for k in keys[-100:]:
    print( sizes[0][k] * k / 1024 / 1024)

In [38]:
m = [ 1,2,3,4,5,6]
m[4:6]

[5, 6]